Analysis

In [648]:
import itertools
import numpy as np
def get_permutations(l,n=3):
    prmts=[]
    for p in itertools. permutations(l, n):
        if sorted(p) not in prmts:
            prmts.append(sorted(p))
    return [ np.asarray(p) for p in prmts if isinstance(p,list) ]
def jsonKeys2int(x):
    if isinstance(x, dict):
            return {int(k):v for k,v in x.items()}
    return x

In [649]:
import pandas as pd

In [684]:
cl=pd.read_json('cl.json').reset_index(drop=True)
cl['sltn']=cl['sltn'].apply(jsonKeys2int)
cl=cl.sort_values('n').reset_index(drop=True)

DD analysis

In [685]:
pd.set_option('display.max_rows', 100)
pd.set_option('display.max_colwidth',500)

In [810]:
def extract(l,x):
    for i in range(5):
        try:
            l.remove(x)
        except:
            break
    return l

def remove_list(l,pr):
    for x in pr:
        try:
            l=extract(l,x)
        except:
            pass
    return l

def remove(l,pr):
    if pr:
        pr=np.concatenate(pr).ravel()
    l=remove_list(l,pr)
    return l

def get_massless_fermions(l,ν,m=0):
    s=-(ν+m)
    l=extract(l,ν)
    if m:
        l=extract(l,m)
    #Dirac masses
    for ss in [s,-s]:
        pr=[ p for  p in get_permuations( l,n=2  ) if np.append( p,ss).sum()==0 ]
        l=remove(l,pr)

    #Majorana masses
    for ss in [s,-s]:
        pr=[ll for ll in l if 2*ll+ss==0]
        l=remove_list(l,pr)
    return list(np.unique(l))

def count_massless_fermions(l,ν,m=0):
    return len( get_massless_fermions(l,ν,m) )

In [811]:
l=[2, -3, -10, 13, 13, -15]
l=[3, -4, -21, 25, 25, -28]
l=[8, 9, 11, -13, -20, -20, 25]
l=[2, 2, -3, 12, -14, -18, 19]
l=[5, 5, 7, -8, -12, -12, 15]
#l=[6, 8, 11, -14, -14, -18, 21]
#l=[3, 3, -8, -15, 20, 20, -23]
#l=[1, -3, 6, -7, -14, 21, 21, -25]
#l=[1, 1, -4, -5, 9, 9, 9, -10, -10]
l=[3, 3, -6, -9, 10, 12, -15, -15, 17]
l=[1, 3, -7, -9, 13, 24, -27, -27, 29]
#XD
l=[2, -5, -5, -5, 7, 8, 8, -10]

In [812]:
get_massless_fermions([2, -3, -3, -5, 9, -13, 15, 16, -18],ν=-3)

[9]

In [813]:
count_massless_fermions( [2, -5, -5, -5, 7, 8, 8, -10],ν=8,m=0)==2

True

In [814]:
count_massless_fermions( [2, -5, -5, -5, 7, 8, 8, -10],ν=8,m=-5)==1

True

In [815]:
count_massless_fermions( [2, -3, -10, 13, 13, -15], ν=13)==0

True

In [816]:
count_massless_fermions( [5, 13, 13, -16, -18, -23, 26],ν=13)==3

True

In [817]:
def massfer(row,label='DarkDirac',nmax=10):
    # 
    if label=='DarkDirac':
        if row.get(label)>0:
            cdd=[]
            for ν in row.get('DD'):
                l=row.get('solution').copy()
                cdd.append( count_massless_fermions(l , ν)  )
            return min(cdd)
        else:
            return nmax
    if label=='XDirac':
        if row.get(label)>0:
            cddmax=100
            for m in row['sltn'].get(3):
                cdp=[]
                for ν in row.get('XD'):
                    l=row.get('solution').copy()
                    l=[ll for ll in l if ll!=m]
                    cdp.append( count_massless_fermions(l , ν,m)  )
                if min(cdp)<cddmax:
                    cdd=cdp
            return min(cdd)                
        else:
            return nmax        

In [818]:
row=cl.loc[2806].to_dict()

In [819]:
label='XDirac'
if True:
        if row.get(label)>0:
            cddmax=100
            for m in row['sltn'].get(3):
                cdp=[]
                for ν in row.get('XD'):
                    l=row.get('solution').copy()
                    l=[ll for ll in l if ll!=m]
                    cdp.append( count_massless_fermions(l , ν,m)  )
                if min(cdp)<cddmax:
                    cdd=cdp
            print( min(cdd)  )
        else:
            print(  nmax    )

0


In [821]:
cl['DD_massless']=cl.apply(lambda row: massfer(row,label='DarkDirac'),axis='columns')

In [822]:
cl['XD_massless']=cl.apply(lambda row: massfer(row,label='XDirac'),axis='columns')

In [823]:
cl[cl.apply(lambda row: massfer(row,label='XDirac'),axis='columns')<10]

,k,l,n,solution,gcd,nu_R,DarkDirac,DarkMajor,XDirac,XMajor,sltn,DD,DM,XD,XM,DD_massless,total,XD_massless
563,"[10, -8, 4]","[10, 4, -6]",8,"[2, -5, -7, -7, 10, 10, 10, -13]",2112,"[{'D→nu_R+2fi': [10, -5]}, {'X→nu_R+fi+fj': [-7, -5, 2]}]",0,1,1,0,"{2: [-7], 3: [10]}",0,[10],[-7],0,10,64,1
652,"[10, -5, 5]","[5, 0, -5]",8,"[2, -5, -5, -5, 7, 8, 8, -10]",625,"[{'D→nu_R+fi+fj': [-10, 2, 8]}, {'X→nu_R+fi+fj': [-10, 7, 8]}]",1,0,1,0,"{2: [8], 3: [-5]}",[8],0,[8],0,2,50,1
696,"[-7, -5, -7]","[-7, 0, 7]",8,"[7, -10, -11, -11, 14, 14, 14, -17]",392,"[{'X→nu_R+fi+fj': [-11, -10, 7]}]",0,0,1,0,"{2: [-11], 3: [14]}",0,0,[-11],0,10,98,1
697,"[1, -4, 1]","[1, 0, 1]",8,"[7, 7, 7, 8, -13, -22, -22, 28]",1,"[{'X→nu_R+fi+fj': [-22, -13, 28]}]",0,0,1,0,"{2: [-22], 3: [7]}",0,0,[-22],0,10,114,1
700,"[-8, -4, -8]","[2, -8, 10]",8,"[1, 1, 1, -4, -11, 14, 14, -16]",720,"[{'X→nu_R+fi+fj': [-11, -4, 14]}]",0,0,1,0,"{2: [14], 3: [1]}",0,0,[14],0,10,62,1
701,"[1, -1, -5]","[1, 0, 1]",8,"[1, 1, 1, -5, -7, 11, 11, -13]",2,"[{'X→nu_R+fi+fj': [-7, -5, 11]}]",0,0,1,0,"{2: [11], 3: [1]}",0,0,[11],0,10,50,1
702,"[3, 7, 3]","[17, 0, 17]",8,"[3, 3, 3, -7, 17, -23, -23, 27]",4624,"[{'X→nu_R+fi+fj': [-23, -7, 27]}]",0,0,1,0,"{2: [-23], 3: [3]}",0,0,[-23],0,10,106,1
703,"[3, -1, 1]","[1, 2, -1]",8,"[1, 3, 3, 3, -5, -7, -7, 9]",2,"[{'X→nu_R+fi+fj': [-7, -5, 9]}]",0,0,1,0,"{2: [-7], 3: [3]}",0,0,[-7],0,10,38,1
704,"[6, 10, -10]","[-10, 0, -2]",8,"[5, -9, -11, -11, 15, 15, 15, -19]",320,"[{'X→nu_R+fi+fj': [-11, -9, 5]}]",0,0,1,0,"{2: [-11], 3: [15]}",0,0,[-11],0,10,100,1
705,"[-6, 4, 9]","[-6, 18, -6]",8,"[1, -6, -6, -6, 9, 11, 11, -14]",1872,"[{'X→nu_R+fi+fj': [-14, 9, 11]}]",0,0,1,0,"{2: [11], 3: [-6]}",0,0,[11],0,10,64,1


In [728]:
cl['total']=cl['solution'].reset_index(drop=True).apply(lambda l: np.abs( np.array(l) ).sum() )

In [734]:
cl[ (cl['n']==9) &  (cl['DarkDirac']>0) ].sort_values(['DD_massless','total'])

,k,l,n,solution,gcd,nu_R,DarkDirac,DarkMajor,XDirac,XMajor,sltn,DD,DM,XD,XM,DD_massless,total
3735,"[4, 8, 6, 3]","[-2, 3, 0]",9,"[1, 1, 2, 2, 3, -5, -6, -6, 8]",4,"[{'D→nu_R+fi+fj': [-5, 2, 3]}, {'D→nu_R+2fi': [-6, 3]}]",1,1,0,0,"{2: [1, 2, -6]}",[2],[-6],0,0,0,34
3388,"[1, 0, -5, 9]","[-3, -1, -2]",9,"[1, -2, 3, 4, 6, -7, -7, -7, 9]",2,"[{'D→nu_R+fi+fj': [-7, -2, 9]}, {'D→nu_R+fi+fj': [-7, 1, 6]}, {'D→nu_R+fi+fj': [-7, 3, 4]}]",3,0,0,0,{3: [-7]},[-7],0,0,0,0,46
3334,"[4, 10, 14, 3]","[16, 2, 10]",9,"[1, -3, -3, -4, -5, 8, 9, 9, -12]",1272,"[{'D→nu_R+fi+fj': [-5, -3, 8]}, {'D→nu_R+fi+fj': [-5, -4, 9]}]",2,0,0,0,"{2: [-3, 9]}","[-3, 9]",0,0,0,0,54
3412,"[-1, 2, 0, -2]","[-9, 5, 1]",9,"[1, 1, -4, -5, 9, 9, 9, -10, -10]",2,"[{'D→nu_R+fi+fj': [-10, 1, 9]}, {'D→nu_R+fi+fj': [-5, -4, 9]}, {'X→nu_R+2fi': [1, -5]}]",2,0,0,1,"{2: [1, -10], 3: [9]}",[9],0,0,[1],0,58
3672,"[1, 12, 0, -3]","[1, 0, -1]",9,"[1, 1, -3, -8, 11, 11, 11, -12, -12]",14,"[{'D→nu_R+fi+fj': [-12, 1, 11]}, {'D→nu_R+fi+fj': [-8, -3, 11]}]",2,0,0,0,"{2: [1, -12], 3: [11]}",[11],0,0,0,0,70
3224,"[4, -2, 0, 11]","[10, 6, -4]",9,"[4, -5, -5, -6, -6, 11, 11, 11, -15]",392,"[{'D→nu_R+fi+fj': [-6, -5, 11]}, {'D→nu_R+fi+fj': [-15, 4, 11]}]",2,0,0,0,"{2: [-5, -6], 3: [11]}",[11],0,0,0,0,74
2008,"[4, -3, -4, -1]","[-4, 0, 16]",9,"[1, -3, 4, 9, 12, -13, -13, -13, 16]",48,"[{'D→nu_R+fi+fj': [-13, -3, 16]}, {'D→nu_R+fi+fj': [-13, 1, 12]}, {'D→nu_R+fi+fj': [-13, 4, 9]}]",3,0,0,0,{3: [-13]},[-13],0,0,0,0,84
3088,"[-5, 0, -10, -7]","[10, 16, 5]",9,"[2, -5, -5, -7, -10, 15, 15, 16, -21]",605,"[{'D→nu_R+fi+fj': [-10, -5, 15]}]",1,0,0,0,"{2: [-5, 15]}","[-5, 15]",0,0,0,0,96
3408,"[2, -1, 0, -10]","[2, 0, -2]",9,"[1, 2, -5, -10, 15, 15, 15, -16, -17]",28,"[{'D→nu_R+fi+fj': [-17, 2, 15]}, {'D→nu_R+fi+fj': [-16, 1, 15]}, {'D→nu_R+fi+fj': [-10, -5, 15]}]",3,0,0,0,{3: [15]},[15],0,0,0,0,96
3402,"[10, 6, -9, 10]","[10, 18, 14]",9,"[2, -4, 5, 10, 13, -15, -15, -15, 19]",1336,"[{'D→nu_R+fi+fj': [-15, -4, 19]}, {'D→nu_R+fi+fj': [-15, 2, 13]}, {'D→nu_R+fi+fj': [-15, 5, 10]}]",3,0,0,0,{3: [-15]},[-15],0,0,0,0,98


def repeated(f):
    r={}
    r[1]=f
    for ri in range(2,f.size+2):
        r[ri]=r[ri-1][pd.Series(r[ri-1]).duplicated()]
        if ri>2:
            for a in np.unique(r[ri]):
                r[ri-1]=r[ri-1][ r[ri-1]!=a ]
                if len(r[ri-1])>0:
                    r[ri-1][ r[ri-1]!=a ]
        if len(r[ri])<=1:
            r[ri]=r[ri]
            r.pop(1)
            break
    return dict( (k,list(v)) for k,v in r.items() if len(v)!=0 )

In [678]:
cl[2799:2800]['solution'].apply(lambda s: repeated(np.asarray(s)))

2799    {2: [-23], 3: [2]}
Name: solution, dtype: object

In [679]:
r={2: [-23], 3: [2]} 

In [683]:
dict( ((k,v) for k,v in r.items() if k!=3) )

{2: [-23]}

In [645]:
def get_nuR_i(row,i,model,label):
    print( row[model]>0,'**',  row['sltn'].get(i))
    if row[model]>0 and row['sltn'].get(i) and row['nu_R'] and len( [ d.get(label) for d in row['nu_R'] if d.get(label)] )>0:
        M=[m  for m in  row['nu_R'] if m.get(label)]
        nuR=[n for n in row['sltn'][i] for m in M  if n in m.get(label)  ]
        #print(nuR)
        if nuR:
            return list(np.unique(nuR))
    else:
        return []

In [647]:
kk.apply(lambda row: get_nuR_i(row,3,'DarkDirac','D→nu_R+fi+fj'),axis='columns')

True ** None


1515    []
dtype: object

In [601]:
kk.sltn.loc[1542]['4']

[2]

In [640]:
cl[1537:1538]

,k,l,n,solution,gcd,nu_R,DarkDirac,DarkMajor,XDirac,XMajor,sltn,DD,DM,XD,XM
1537,"[6, -1, 7, 1]","[12, 0, 9]",9,"[2, -3, -3, -5, 9, -13, 15, 16, -18]",432,"[{'D→nu_R+fi+fj': [-13, -3, 16]}]",1,0,0,0,{2: [-3]},[-3],0,0,0


In [471]:
cl[:3]

,DarkDirac,DarkMajor,XDirac,XMajor,gcd,k,l,n,nu_R,sltn,solution,DD,DM,XD,XM
0,2,0,0,0,2,"[-5, 0]","[1, -1]",6,"[{'D→nu_R+fi+fj': [-15, 2, 13]}, {'D→nu_R+fi+fj': [-10, -3, 13]}]",{'2': [13]},"[2, -3, -10, 13, 13, -15]",[13],0,0,0
1,2,0,0,0,7749,"[4, 7]","[21, 18]",6,"[{'D→nu_R+fi+fj': [-28, 3, 25]}, {'D→nu_R+fi+fj': [-21, -4, 25]}]",{'2': [25]},"[3, -4, -21, 25, 25, -28]",[25],0,0,0
2,2,0,0,0,1,"[-4, 0]","[-1, 1]",6,"[{'D→nu_R+fi+fj': [-20, 3, 17]}, {'D→nu_R+fi+fj': [-12, -5, 17]}]",{'2': [17]},"[3, -5, -12, 17, 17, -20]",[17],0,0,0


### Tree level neutrino Dirac masses

In [416]:
l=get_massless_fermions( [2, -3, -10, 13, 13, -15], ν=13,m=0)
[ ll for ll in l if ll+ν-s==0]

[]

In [417]:
[ ll for ll in l if ll-ν+s==0]

[]